In [65]:
import io
from typing import Dict, List
from datetime import datetime, timedelta
from pydantic import BaseModel
from time import sleep

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import berserk
import chess, chess.pgn

In [4]:
from dash import Dash, html, dcc

In [50]:
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    name: str
    id: str

class GlobalAnalysis(BaseModel):
    inaccuracy: int
    mistake: int
    blunder: int
    acpl: int

class Player(BaseModel):
    user: User
    rating: int
    ratingDiff: Optional[int] = 0
    analysis: Optional[GlobalAnalysis] = None

class Players(BaseModel):
    white: Player
    black: Player

class Opening(BaseModel):
    eco: str
    name: str
    ply: int
        
class Judgment(BaseModel):
    name: str
    comment: str

class MoveAnalysis(BaseModel):
    mate: Optional[int]
    eval: Optional[int]
    best: Optional[str] = None
    variation: Optional[str] = None
    judgment: Optional[Judgment] = None

class Clock(BaseModel):
    initial: int
    increment: int
    totalTime: int

class GameModel(BaseModel):
    id: str
    rated: bool
    variant: str
    speed: str
    createdAt: datetime
    lastMoveAt: datetime
    status: str
    players: Players
    winner: Optional[str] = 'draw'
    opening: Optional[Opening] = None
    moves: str
    analysis: Optional[List[MoveAnalysis]] = None
    clock: Clock

## Functions

In [51]:
# Read PGN string into python-chess object
def read_pgn(game_str):
    pgn = io.StringIO(game_str)
    game = chess.pgn.read_game(pgn)
    return game

In [52]:
# Read python-chess object, play and display it in them on a board.
def play_game(game):
    board = game.board()
    for move in game.mainline_moves():
        clear_output(wait=True)
        print(board)
        board.push(move)
        sleep(1)

In [53]:
# Function that returns list of position evaluations for entire game.
# Scores are in centipawn and from White perspective
# If value is None it's because engine saw checkmate in #n
def read_evals(game):
    
    eval_lst = []
    for node in game.mainline():
        if node.eval() != None:
            eval_lst.append(node.eval().white().score())

    return eval_lst

def read_moves(game):
    
    move_lst = [str(node.san()) for node in game.mainline()]
    return(move_lst)

def read_game_eval_recursive(game_node):
    
    if game_node.is_end():
        print(game_node.eval())
        return [None]
    else:
        if game_node.eval()!=None:
            return [game_node.eval().white().score()] + read_game_eval(game_node[0])
        else:
            return read_game_eval(game_node[0])   

In [226]:
class lichess_communication:
    
    lichess_id = None
    API_TOKEN = None
    client = None
    get_opening = True
    get_evals = True
    
    games_lst = []
    games_df = None
    
    def __init__(self, user) -> None:
        
        # Initialize token
        with open('conf/token.txt') as f:
            self.API_TOKEN = f.readline()[:-1]
        
        # Initialize lichess client
        self.lichess_id = user
        session = berserk.TokenSession(self.API_TOKEN)
        self.client = berserk.Client(session=session)
    
    def fetch_games_by_dates(self, since: int, until: int) -> List[GamesModel]:
    
        games_gen = self.client.games.export_by_player(self.lichess_id,
                                                       rated=True,
                                                       since=since,
                                                       until=until,
                                                       evals=self.get_evals,
                                                       opening=self.get_opening)
        
        self.games_lst = [GameModel(**game) for game in games_gen] 

        return self.games_lst
    
        #for game in games_gen:
        #    try:
        #        self.games_lst.append(GameModel(**game))
        #    except:
        #        print(game)
        
    def show_games_info(self) -> None:
        
        if self.games_lst == None:
            print('No games have been fetched')
            return False
        else:
            print(f'Fetched {len(self.games_lst)} games')
            print(f'Last game from  : {self.games_lst[0].createdAt}')
            print(f'First game from : {self.games_lst[-1].createdAt}')


    def fill_df(self) -> pd.DataFrame:

        games_dict = dict(lichess_id =[],
                          color = [],
                          oponent = [],
                          time = [],
                          opening = [],
                          result = [],
                          moves = [],
                          analysis = [],
                          evals = [],
                          judgment = [])

        for game in games_lst:

            games_dict['lichess_id'].append(game.id)
            games_dict['opening'].append(game.opening.name)
            games_dict['color'].append('white' if (game.players.white==self.lichess_id) else 'black')

            if  game.winner=='draw':
                games_dict['result'].append('draw')
            elif game.winner=='white' and game.players.white.user.id==self.lichess_id:
                games_dict['result'].append('win')
            elif game.winner=='black' and game.players.black.user.id==self.lichess_id:
                games_dict['result'].append('win')
            else:
                games_dict['result'].append('loss')

            games_dict['time'].append(game.speed)

            if game.players.white.user.id==self.lichess_id:
                games_dict['oponent'].append(game.players.black.user.id)
            else:
                games_dict['oponent'].append(game.players.white.user.id)

            games_dict['moves'].append(str(game.moves))
            
            games_dict['analysis'].append(True if game.analysis!=None else False)
            
            games_dict['evals'].append(str([move_anal.eval for move_anal in game.analysis] 
                                       if game.analysis!=None else None))
            
            games_dict['judgment'].append(str([move_anal.judgment.name if move_anal.judgment!=None else None for move_anal in game.analysis] 
                                          if game.analysis!=None else None))

        self.games_df = pd.DataFrame.from_dict(games_dict, orient='columns')
        
        return self.games_df

In [228]:
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///:memory:')
lichess_comm.games_df.to_sql('game_data', engine)

25

## API game request settings

In [227]:
lichess_id = 'miguel0f'

last_Xdays = datetime.now()-timedelta(days=20)
since = int(berserk.utils.to_millis(last_Xdays))
until = int(berserk.utils.to_millis(datetime.now()))

lichess_comm = lichess_communication(lichess_id)
lichess_comm.fetch_games_by_dates(since,until)
lichess_comm.fill_df()

,lichess_id,color,oponent,time,opening,result,moves,analysis,evals,judgment
0,Vzn667SP,black,branovasil,classical,"Scotch Game: Scotch Gambit, London Defense",win,e4 e5 Nf3 Nc6 d4 exd4 Bc4 Bb4+ c3 dxc3 bxc3 Qf...,True,"[33, 12, 19, 32, 13, 13, 0, -13, 0, 0, 0, 99, ...","[None, None, None, None, None, None, None, Non..."
1,My7vAPF8,black,branovasil,classical,Englund Gambit Complex: Hartlaub-Charlick Gambit,win,d4 e5 dxe5 d6 exd6 Bxd6 c3 Nc6 e4 Qe7 f3 Be6 B...,False,None,None
2,jKKSsOq4,black,aleksanderdaniloff,blitz,Elephant Gambit,win,e4 e5 Nf3 d5 Nxe5 dxe4 Nc3 Nf6 d3 Bb4 Bd2 O-O ...,True,"[33, 12, 19, 123, 76, 82, -57, 89, -19, 0, -48...","[None, None, None, 'Inaccuracy', None, None, '..."
3,6hxDZllQ,black,oknekim,blitz,Elephant Gambit: Paulsen Countergambit,loss,e4 e5 Nf3 d5 exd5 e4 Ng1 Qxd5 Nc3 Qe5 d3 exd3+...,True,"[33, 12, 19, 123, 130, 140, 0, 5, 13, 35, 9, 5...","[None, None, None, 'Inaccuracy', None, None, '..."
4,Bum7UaeP,black,uos,blitz,Russian Game: Stafford Gambit,loss,e4 e5 Nf3 Nf6 Nxe5 Nc6 Nxc6 dxc6 Nc3 Bc5 Bc4 N...,False,None,None
5,yupZEyr2,black,man05060135,rapid,Scotch Game,draw,e4 e5 Nf3 Nc6 d4 d5 Nxe5 dxe4 Nxc6 bxc6 Bc4 c5...,True,"[33, 12, 19, 32, 13, 93, 106, 98, 40, 32, 21, ...","[None, None, None, None, None, 'Inaccuracy', N..."
6,dyQv6av1,black,man05060135,rapid,Elephant Gambit,win,e4 e5 Nf3 d5 Nxe5 dxe4 Nxf7 Kxf7 Bc4+ Ke8 Qh5+...,False,None,None
7,JIToP0iT,black,pawnblusher,rapid,"Sicilian Defense: Smith-Morra Gambit Accepted,...",loss,e4 c5 d4 cxd4 c3 dxc3 Nxc3 Nc6 Nf3 a6 Bc4 e6 O...,True,"[33, 32, 10, 1, -26, -20, -28, -19, -64, -6, -...","[None, None, None, None, None, None, None, Non..."
8,3kMjeJKL,black,e_1_2_3,blitz,Englund Gambit Complex: Hartlaub-Charlick Gambit,loss,d4 e5 dxe5 d6 exd6 Bxd6 Nf3 Nc6 Nc3 Qe7 Nd5 Qe...,False,None,None
9,9mSGJBJQ,black,uglyslug,blitz,Sicilian Defense: Smith-Morra Gambit,win,e4 c5 d4 cxd4 c3 dxc3 Nxc3 Nc6 Nf3 e5 Bc4 Bb4 ...,False,None,None


In [ ]:
games_df = games_dataframe(games_lst, lichess_id)

In [100]:
for k in range(len(lichess_comm.games_lst)):
    speed = lichess_comm.games_lst[k].speed
    pref = lichess_comm.games_lst[k].perf
    if speed !=  pref:
        print(speed, pref)

classical fromPosition
classical fromPosition
classical fromPosition


In [94]:
for k in range(len(lichess_comm.games_lst)):
    opponent = lichess_comm.games_lst[k].players.black.user.id
    if opponent == 'man05060135':
        print(lichess_comm.games_lst[k])

id='yupZEyr2' rated=True variant='standard' speed='rapid' perf='rapid' createdAt=datetime.datetime(2022, 7, 18, 23, 4, 35, 34000, tzinfo=datetime.timezone.utc) lastMoveAt=datetime.datetime(2022, 7, 18, 23, 46, 55, 91000, tzinfo=datetime.timezone.utc) status='draw' players=Players(white=Player(user=User(name='miguel0f', id='miguel0f'), rating=1693, ratingDiff=-1, analysis=GlobalAnalysis(inaccuracy=10, mistake=3, blunder=4, acpl=55)), black=Player(user=User(name='Man05060135', id='man05060135'), rating=1672, ratingDiff=1, analysis=GlobalAnalysis(inaccuracy=4, mistake=4, blunder=6, acpl=57))) winner='draw' opening=Opening(eco='C44', name='Scotch Game', ply=5) moves='e4 e5 Nf3 Nc6 d4 d5 Nxe5 dxe4 Nxc6 bxc6 Bc4 c5 Bb5+ Ke7 d5 a6 Bc6 Rb8 O-O Bb7 Re1 f5 Nc3 Bxc6 dxc6 Qxd1 Rxd1 Nf6 Bf4 h6 Bxc7 Rc8 Bd6+ Kf7 c7 Bxd6 Rxd6 Rxc7 Rxa6 Rb7 b3 Rhb8 Rd1 Ne8 Kf1 Kf8 Rd5 Nc7 Rxf5+ Kg8 Ra5 Ne6 Re5 Nd4 Nxe4 Nxc2 Raxc5 Nd4 f4 Rf8 g3 Rd7 b4 Nf3 Kg2 Nxe5 Rxe5 Rdd8 a4 Rd4 b5 Rxa4 b6 Ra2+ Kh3 Rb8 Re6 Rb2 f5 R2x

In [177]:
print(type(lichess_comm.games_lst[0]))
[move.eval for move in lichess_comm.games_lst[0].analysis]

<class '__main__.GameModel'>


[33,
 12,
 19,
 32,
 13,
 13,
 0,
 -13,
 0,
 0,
 0,
 99,
 175,
 97,
 112,
 774,
 805,
 825,
 822,
 796,
 395,
 695,
 716,
 738,
 635,
 670,
 672,
 889,
 711,
 717,
 620,
 753,
 717,
 837,
 838,
 933,
 882,
 934,
 871,
 885,
 777,
 1053,
 1023,
 1533,
 966,
 962,
 878,
 1110,
 1123,
 2345,
 2281,
 2388,
 2452]

In [85]:
len(lichess_comm.games_lst)

565

In [ ]:
games_df['result'].value_counts()

In [ ]:
games_df.loc[games_df['result']=='loss']

In [ ]:
game = read_pgn(games_lst[12])
game_eval = games_df.iloc[12]
play_game(game)

plt.figure()
plt.plot(game_eval)
plt.show()

## Code snippets to look into data returned by API

In [ ]:
for k in range(len(a)):
    if a[k]['players'][color]['user']['name']!='miguel0f':
        print('ahahah')